Imports the needed Python libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import metrics 
from sklearn import linear_model
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Reads in the data sets

In [2]:
x_test = pd.read_csv('x_test.csv')
x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')

FileNotFoundError: ignored

Normalizes the data with the Standard Scaler from Sklearn

In [ ]:
x_train[x_train.columns] = StandardScaler().fit_transform(x_train[x_train.columns])

Drops columns that are highly correlated with other columns, correlations threshold of 0.6, and favors removing higher number columns.

In [ ]:
for column in x_train.columns[::-1]:
    cor_target = abs(x_train.corr()[column])
    rev_preds = cor_target[cor_target>0.6]
    print(len(x_train.columns))
    for i in range(len(rev_preds.index)):
        col2remove = rev_preds.index[i]
        if column != col2remove and int(column[-2:].strip()) < int(rev_preds.index[i][-2:].strip()):
            x_train = x_train.drop(columns=[col2remove])


Chooses the k best predictors for all k and then calculates the R^2, AIC, and BIC for the resulting models and finds the best one according to each criterion.

In [ ]:
adjr2lst= []
best_amt_preds = 0
AIClst= [100000]
best_amt_preds_AIC = 0
BIClst= [100000]
best_amt_preds_BIC = 0


adjr2lassolst= []
best_amt_preds_lasso = 0
AIClassolst= [100000]
best_amt_preds_lasso_AIC = 0
BIClassolst= [100000]
best_amt_preds_lasso_BIC = 0
for krange in range(1,len(x_train.columns)+1):
    pred_select = SelectKBest(score_func=f_regression, k=krange)
    fit_pred_select = pred_select.fit_transform(x_train, y_train['Output'])
    print(krange)
    pred_support_list = pred_select.get_support()

    best_model_x_train = pd.DataFrame({})
    for i in range(len(pred_support_list)):
        if pred_support_list[i] == True:
            for j in range(len(x_train.columns)):
                if i==j:
                    best_model_x_train['{}'.format(x_train.columns[j])] = x_train[x_train.columns[j]]

    reg= linear_model.LinearRegression().fit(best_model_x_train,y_train['Output'])
    y_pred = reg.predict(best_model_x_train)
    adjr2 = 1-(1-metrics.r2_score(y_train['Output'], y_pred))*(len(best_model_x_train)-1)/(len(best_model_x_train)- len(best_model_x_train.columns)-1)
    sse=0
    for i in range(len(y_pred)):
        sse += (y_pred[i] - y_train['Output'][i])**2
    AIC = 2*len(best_model_x_train.columns) + len(best_model_x_train)*np.log(sse/(len(best_model_x_train) - len(best_model_x_train.columns)))
    BIC = len(best_model_x_train)*np.log(sse) + len(best_model_x_train.columns)*np.log(sse/(len(best_model_x_train) - len(best_model_x_train.columns)))


    reglasso = linear_model.Lasso().fit(best_model_x_train,y_train['Output'])
    y_pred_lasso = reglasso.predict(best_model_x_train)
    adjr2lasso = 1-(1-metrics.r2_score(y_train['Output'], y_pred_lasso))*(len(best_model_x_train)-1)/(len(best_model_x_train)- len(best_model_x_train.columns)-1)
    sselasso = 0
    for i in range(len(y_pred)):
        sselasso += (y_pred_lasso[i] - y_train['Output'][i])**2
    AIClasso = 2*len(best_model_x_train.columns) + len(best_model_x_train)*np.log(sselasso/(len(best_model_x_train) - len(best_model_x_train.columns)))
    BIClasso = len(best_model_x_train)*np.log(sselasso) + len(best_model_x_train.columns)*np.log(sselasso/(len(best_model_x_train) - len(best_model_x_train.columns)))

    adjr2lassolst.append(adjr2lasso)
    if adjr2lasso >= max(adjr2lassolst):
        new_x_train_lasso = best_model_x_train
        best_amt_preds_lasso = krange
    print('Adj. R^2 Lasso:', adjr2lasso)

    adjr2lst.append(adjr2)
    if adjr2 >= max(adjr2lst):
        new_x_train = best_model_x_train
        best_amt_preds = krange
    print('Adj. R^2:', adjr2)
    print('------------\n')


    if AIClasso <= min(AIClassolst) and krange > 1:
        AIClassolst.append(AIClasso)
        new_x_train_lasso_AIC = best_model_x_train
        best_amt_preds_lasso_AIC = krange
    print('AIC Lasso:', AIClasso)


    if AIC <= min(AIClst) and krange > 1:
        AIClst.append(AIC)
        new_x_train_AIC = best_model_x_train
        best_amt_preds_AIC = krange
    print('AIC:', AIC)
    print('------------\n')

    if BIClasso <= min(BIClassolst) and krange > 1:
        BIClassolst.append(BIClasso)
        new_x_train_lasso_BIC = best_model_x_train
        best_amt_preds_lasso_BIC = krange
    print('BIC Lasso:', BIClasso)

    if BIC <= min(BIClst) and krange > 1:
        BIClst.append(BIC)
        new_x_train_BIC = best_model_x_train
        best_amt_preds_BIC = krange
    print('BIC:', BIC)
    print('------------\n')

print(max(adjr2lst))
print(best_amt_preds)
print(max(adjr2lassolst))
print(best_amt_preds_lasso)

print(min(AIClst))
print(best_amt_preds_AIC)
print(min(AIClassolst))
print(best_amt_preds_lasso_AIC)

print(min(BIClst))
print(best_amt_preds_BIC)
print(min(BIClassolst))
print(best_amt_preds_lasso_BIC)

Finds the mean MSE of many models over a 1000 trials of by spliting the training data given into training and testing data.  Then calculates the average MSE of the best models produced in the last section to find the best model at predicting.

In [ ]:
n= 1000
for j in range(1,n+1):

    leanModelReg = linear_model.LinearRegression()
    X_train2, X_test2, y_train2, y_test2 = train_test_split(new_x_train, y_train['Output'], test_size = .3, random_state = j)
    leanModelReg.fit(X_train2,y_train2)
    y_pred2 = leanModelReg.predict(X_test2)
    leanMSEtotal += metrics.mean_squared_error(y_test2,y_pred2)
    #print(leanModelReg.score(X_test2,y_test2))
    #print('MSE:', metrics.mean_squared_error(y_test2,y_pred2))

    leanModelRegAIC = linear_model.LinearRegression()
    X_train2, X_test2, y_train2, y_test2 = train_test_split(new_x_train_AIC, y_train['Output'], test_size = .3, random_state = j)
    leanModelRegAIC.fit(X_train2,y_train2)
    y_pred2 = leanModelRegAIC.predict(X_test2)
    leanMSEtotalAIC += metrics.mean_squared_error(y_test2,y_pred2)



    for i in range(1,11):
        #print('alpha=',i*0.1)
        lassoModelReg = linear_model.Lasso(alpha = 0.3*i,max_iter=10000)
        X_train2, X_test2, y_train2, y_test2 = train_test_split(new_x_train_lasso, y_train['Output'], test_size = .3,random_state = j)
        lassoModelReg.fit(X_train2,y_train2)
        y_pred2 = lassoModelReg.predict(X_test2)
        #print(lassoModelReg.score(X_test2,y_test2))
        lassoMSEtotal[i-1] += metrics.mean_squared_error(y_test2,y_pred2)
        #print('MSE:', metrics.mean_squared_error(y_test2,y_pred2))


    for i in range(1,11):
        #print('alpha=',i*0.1)
        lassoModelRegAIC = linear_model.Lasso(alpha = 0.1*i)
        X_train2, X_test2, y_train2, y_test2 = train_test_split(new_x_train_lasso_AIC, y_train['Output'], test_size = .3,random_state = j)
        lassoModelRegAIC.fit(X_train2,y_train2)
        y_pred2 = lassoModelRegAIC.predict(X_test2)
        #print(lassoModelReg.score(X_test2,y_test2))
        lassoMSEtotalAIC[i-1] += metrics.mean_squared_error(y_test2,y_pred2)
        #print('MSE:', metrics.mean_squared_error(y_test2,y_pred2))

    fullModelReg = linear_model.LinearRegression()
    X_train2, X_test2, y_train2, y_test2 = train_test_split(x_train, y_train['Output'], test_size = .3, random_state = j)
    fullModelReg.fit(X_train2,y_train2)
    y_pred2 = fullModelReg.predict(X_test2)
    #print(fullModelReg.score(X_test2,y_test2))
    semifullMSEtotal += metrics.mean_squared_error(y_test2,y_pred2)
    #print('MSE:', metrics.mean_squared_error(y_test2,y_pred2))

print(leanMSEtotal/n)
lassoMSEtotal[:] = [(x / n) for x in lassoMSEtotal]
print(lassoMSEtotal)
print(semifullMSEtotal/n)

print(leanMSEtotalAIC/n)
lassoMSEtotalAIC[:] = [(x / n) for x in lassoMSEtotalAIC]
print(lassoMSEtotalAIC)

Creates a new model using the parameters that gave the lowest mean MSE (Lasso, alpha 0.5, AIC criterion)

In [ ]:
allDataLeanReg = linear_model.Lasso(alpha=0.5,max_iter=1000000,tol=1e-6)
allDataLeanReg.fit(new_x_train_lasso_AIC,y_train['Output'])

Drops the unnescesary predictors from the x_test

In [ ]:
for column in x_test.columns:
    if not (column in new_x_train_lasso_AIC.columns):
        x_test = x_test.drop(columns=column)

Standardizes x_test data with Sklearn's Standard Scaler

In [ ]:
x_test[x_test.columns] = StandardScaler().fit_transform(x_test[x_test.columns])

Writes the data to pandas data frame and writes to resulting csv file.

In [ ]:
Submission = pd.DataFrame({
    'Sl. No.':range(1,201),
    'Output':y_pred
})
Submission.to_csv('Submission_Project1_KeithSimmons',index=False)